#### unitycatalog-litellm E2E example
Import and run this notebook in Databricks

In [0]:
%pip install litellm unitycatalog-ai unitycatalog-client unitycatalog-litellm -q -U
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


#### Create a DatabricksFunctionClient
Use serverless client here as an example

In [0]:
import os

assert "OPENAI_API_KEY" in os.environ, (
    "Please set the OPENAI_API_KEY environment variable to your OpenAI API key"
)

In [0]:
from unitycatalog.ai.core.base import set_uc_function_client
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

client = DatabricksFunctionClient()

# set the default uc function client, it will be used for all toolkits
set_uc_function_client(client)

#### Create a UC function for executing python code

In [0]:
# replace with your own catalog and schema
CATALOG = "main"
SCHEMA = "default"


def execute_python_code(code: str) -> str:
    """
    Executes the given python code and returns its stdout.

    Args:
      code: Python code to execute. Remember to print the final result to stdout.
    """
    import sys
    from io import StringIO

    stdout = StringIO()
    sys.stdout = stdout
    exec(code)
    return stdout.getvalue()


function_info = client.create_python_function(
    func=execute_python_code, catalog=CATALOG, schema=SCHEMA, replace=True
)
python_execution_function_name = function_info.full_name

#### Create a UC function for translation

Use Databricks built-in [ai_translate](https://docs.databricks.com/en/sql/language-manual/functions/ai_translate.html) function

In [0]:
translate_function_name = f"{CATALOG}.{SCHEMA}.translate"
sql_body = f"""CREATE FUNCTION {translate_function_name}(content STRING COMMENT 'the text to be translated', language STRING COMMENT 'the target language code to translate the content to')
RETURNS STRING
RETURN SELECT ai_translate(content, language)
"""
client.create_function(sql_function_body=sql_body)

/local_disk0/.ephemeral_nfs/envs/pythonEnv-ac15189f-1e8b-4bba-b9f2-601de000d4e5/lib/python3.10/site-packages/unitycatalog/ai/core/databricks.py:298: UserWarning: The function translate does not have a description. Using Unity Catalog functions that do not have function descriptions limits the functionality for an LLM to understand when it is appropriate to call your function as a tool and how to properly interface with the function. Update your function's description with a verbose entry in the 'comments' parameter to improve the usage characterstics of this function as a tool.
  check_function_info(created_function_info)


FunctionInfo(catalog_name='main', comment=None, created_at=1734104011051, created_by='michael.berk@databricks.com', data_type=<ColumnTypeName.STRING: 'STRING'>, external_language=None, external_name=None, full_data_type='STRING', full_name='main.default.translate', function_id='2c244f6f-61b1-4fc3-9bc6-dbe5e8daea0a', input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='content', type_text='string', type_name=<ColumnTypeName.STRING: 'STRING'>, position=0, comment='the text to be translated', parameter_default=None, parameter_mode=None, parameter_type=<FunctionParameterType.PARAM: 'PARAM'>, type_interval_type=None, type_json='{"name":"content","type":"string","nullable":true,"metadata":{"comment":"the text to be translated"}}', type_precision=0, type_scale=0), FunctionParameterInfo(name='language', type_text='string', type_name=<ColumnTypeName.STRING: 'STRING'>, position=1, comment='the target language code to translate the content to', parameter_default=None, param

In [0]:
client.execute_function(
    translate_function_name, {"content": "What is Databricks?", "language": "es"}
)

/local_disk0/.ephemeral_nfs/envs/pythonEnv-ac15189f-1e8b-4bba-b9f2-601de000d4e5/lib/python3.10/site-packages/unitycatalog/ai/core/databricks.py:581: UserWarning: The function translate does not have a description. Using Unity Catalog functions that do not have function descriptions limits the functionality for an LLM to understand when it is appropriate to call your function as a tool and how to properly interface with the function. Update your function's description with a verbose entry in the 'comments' parameter to improve the usage characterstics of this function as a tool.
  check_function_info(function_info)


FunctionExecutionResult(error=None, format='SCALAR', value='¿Qué es Databricks?', truncated=None)

#### Create tools

In [0]:
from unitycatalog.ai.litellm.toolkit import UCFunctionToolkit

toolkit = UCFunctionToolkit(
    function_names=[python_execution_function_name, translate_function_name]
)
tools = toolkit.tools
tools

/home/spark-ac15189f-1e8b-4bba-b9f2-60/.ipykernel/14958/command-1819603593741393-926642350:3: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  toolkit = UCFunctionToolkit(


[LiteLLMTool(name='main__default__execute_python_code', description='Executes the given python code and returns its stdout.', tool={'type': 'function', 'function': {'name': 'main__default__execute_python_code', 'strict': True, 'parameters': {'properties': {'code': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'description': 'Python code to execute. Remember to print the final result to stdout.', 'title': 'Code'}}, 'title': 'main__default__execute_python_code__params', 'type': 'object', 'additionalProperties': False, 'required': ['code']}, 'description': 'Executes the given python code and returns its stdout.'}}),
 LiteLLMTool(name='main__default__translate', description='', tool={'type': 'function', 'function': {'name': 'main__default__translate', 'strict': True, 'parameters': {'properties': {'content': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'description': 'the text to be translated', 'title': 'Content'}, 'language': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'des

#### Use the tools in LiteLLM Agent

In [0]:
import litellm

# Define your request
question = "What is 'cerdo hormiguero' in english?"
messages = [{"role": "user", "content": question}]

# Show the response
response = litellm.completion(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
)
response

 ModelResponse(id='chatcmpl-Ae2IAKtJJzP0P3ynKkCudoBhNcJpz', created=1734105294, model='gpt-4o-mini-2024-07-18', object='chat.completion', system_fingerprint='fp_6fc10e10eb', choices=[Choices(finish_reason='tool_calls', index=0, message=Message(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(function=Function(arguments='{"content":"cerdo hormiguero","language":"en"}', name='main__default__translate'), id='call_Vz5ZG9JUgudEv5e1gM4o2wVg', type='function')], function_call=None))], usage=Usage(completion_tokens=25, prompt_tokens=152, total_tokens=177, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier=None)


#### Leverage `generate_tool_call_messages` to run the UC Function

In [0]:
from unitycatalog.ai.litellm.utils import generate_tool_call_messages

tool_messages = generate_tool_call_messages(
    response=response, client=client, conversation_history=messages
)

response_2 = litellm.completion(
    model="gpt-4o-mini",
    messages=tool_messages,
)
response_2

/local_disk0/.ephemeral_nfs/envs/pythonEnv-ac15189f-1e8b-4bba-b9f2-601de000d4e5/lib/python3.10/site-packages/unitycatalog/ai/core/databricks.py:581: UserWarning: The function translate does not have a description. Using Unity Catalog functions that do not have function descriptions limits the functionality for an LLM to understand when it is appropriate to call your function as a tool and how to properly interface with the function. Update your function's description with a verbose entry in the 'comments' parameter to improve the usage characterstics of this function as a tool.
  check_function_info(function_info)



 ModelResponse(id='chatcmpl-Ae2IDXCqk5xJPOJzhGjiY7MN6ykGN', created=1734105297, model='gpt-4o-mini-2024-07-18', object='chat.completion', system_fingerprint='fp_6fc10e10eb', choices=[Choices(finish_reason='stop', index=0, message=Message(content='The term "cerdo hormiguero" translates to "ant bear" in English, which is commonly known as the "anteater."', role='assistant', tool_calls=None, function_call=None))], usage=Usage(completion_tokens=28, prompt_tokens=57, total_tokens=85, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier=None)
